In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [18]:
import os
key=os.getenv('OPENAI_API_KEY')

prompt="please book 2 ticket rajimurugan movie"


tools = [
        {
            "type": "function",
            "function": {
                "name": "booktickets",
                "description": "book movie tickets ",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "movie_name":{
                            "type": "string"
                        },
                        "tickets":{
                            "type": "string"
                        }  
                    },
                    "required": ["movie_name","tickets"],
                },
            },
        }
    ]

In [9]:
import openai
openai.api_key=key

In [19]:
def booktickets(movie_name,tickets):
    return f"you have booked {tickets} tickets for {movie_name} movie"

response=openai.chat.completions.create(
     model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are a helpful assistant to handle the booking system."},
            {"role": "user", "content": prompt}
        ],
        tools=tools,
        tool_choice="auto"
)

In [37]:
json_bookdata=response.choices[0].message.tool_calls[0].function.arguments

In [35]:
import json

In [44]:
booking_details=json.loads(json_bookdata)
print(booking_details)


{'movie_name': 'rajimurugan', 'tickets': '2'}


In [45]:
type(booking_details)

dict

In [7]:
from openai import OpenAI
import json

client = OpenAI(api_key=key)

# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    if "tokyo" in location.lower():
        return json.dumps({"location": "Tokyo", "temperature": "10", "unit": unit})
    elif "san francisco" in location.lower():
        return json.dumps({"location": "San Francisco", "temperature": "72", "unit": unit})
    elif "paris" in location.lower():
        return json.dumps({"location": "Paris", "temperature": "22", "unit": unit})
    else:
        return json.dumps({"location": location, "temperature": "unknown"})

def run_conversation():
    # Step 1: send the conversation and available functions to the model
    messages = [{"role": "user", "content": "What's the weather like in San Francisco, Tokyo, and Paris?"}]
    tools = [
        {
            "type": "function",
            "function": {
                "name": "get_current_weather",
                "description": "Get the current weather in a given location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city and state, e.g. San Francisco, CA",
                        },
                        "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                    },
                    "required": ["location"],
                },
            },
        }
    ]
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        tools=tools,
        tool_choice="auto",  # auto is default, but we'll be explicit
    )
    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls
    # Step 2: check if the model wanted to call a function
    if tool_calls:
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        available_functions = {
            "get_current_weather": get_current_weather,
        }  # only one function in this example, but you can have multiple
        messages.append(response_message)  # extend conversation with assistant's reply
        # Step 4: send the info for each function call and function response to the model
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            function_response = function_to_call(
                location=function_args.get("location"),
                unit=function_args.get("unit"),
            )
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )  # extend conversation with function response
        second_response = client.chat.completions.create(
            model="gpt-4o",
            messages=messages,
        )  # get a new response from the model where it can see the function response
        return second_response
print(run_conversation())

ChatCompletion(id='chatcmpl-9eQgLsa9JzGI7dLGcKo40srYKKHKM', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Here are the current weather conditions:\n\n- **San Francisco, CA**: 22°C\n- **Tokyo, Japan**: 10°C\n- **Paris, France**: 22°C\n\nIf you need more details or an update, feel free to ask!', role='assistant', function_call=None, tool_calls=None))], created=1719422713, model='gpt-4o-2024-05-13', object='chat.completion', system_fingerprint='fp_4008e3b719', usage=CompletionUsage(completion_tokens=55, prompt_tokens=173, total_tokens=228))
